In [1]:
# collecting and dividing the data

from PIL import Image
import PIL
import os
dirname = "Warwick QU Dataset (Released 2016_07_08)/"
reshapedirname = "reshaped_warwick/"
final_height = 400
final_width = 600
try:
	os.stat(reshapedirname)
except:
	os.mkdir(reshapedirname)
for subdirname in ["segments_train","segments_test","segments_train_eval","images_train","images_test","images_train_eval"]:
	try:
		os.stat(reshapedirname+subdirname)
	except:
		os.mkdir(reshapedirname+subdirname)
prefix_name = ["images","segments"]
suffix_name = ["train","test"]
for file in os.listdir(dirname):
	if file.endswith(".bmp"):
		img = Image.open(dirname+file)
		width, height = img.size
		istest,isanno = False,False
		if file[:4] == "test":
			istest = True
		if file.find("anno") != -1:
			isanno = True
		if isanno:
			# change the image to saturated
			img = img.point(lambda i: i * 255)
		reshaped_image = img = img.resize((final_width, final_height), PIL.Image.ANTIALIAS)
		newfilename = reshapedirname+prefix_name[isanno]+"_"+suffix_name[istest]+"/"+file.split('.')[0]
		newfilenamefull = newfilename+".png"
		reshaped_image.save(newfilenamefull,"PNG")

In [2]:
# Routine for decoding the Dataset binary file format

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import tensorflow as tf
import random
import pandas as pd
import numpy as np

# Process images of this size.
IMAGE_WIDTH = 600
IMAGE_HEIGHT = 400

# Global constants describing the Dataset data set.
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 320
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 20


def read_from_queue(path,num_channels):
    return tf.image.decode_png(path, channels=0, dtype=tf.uint16)


def read_dataset(all_files_queue):

    class DatasetRecord(object):
        pass

    result = DatasetRecord()

    # Read a record, getting filenames from the filename_queue.
    text_reader = tf.TextLineReader()
    _, csv_content = text_reader.read(all_files_queue)

    i_path, s_path = tf.decode_csv(csv_content,
                                           record_defaults=[[""], [""]])

    result.uint8image = read_from_queue(tf.read_file(i_path),0)
    segment = read_from_queue(tf.read_file(s_path),0)

    result.label = segment
    result.i_path = i_path
    result.s_path = s_path
    result.csv = csv_content
    return result


def _generate_image_and_label_batch(image, label, i_path, min_queue_examples,
                                    batch_size, shuffle):
    
    # Create a queue that shuffles the examples, and then
    # read 'batch_size' images + labels from the example queue.
    num_preprocess_threads = 16
    images, labels, i_paths = tf.train.batch(
        [image, label, i_path],
        batch_size=batch_size,
        num_threads=num_preprocess_threads,
        capacity=min_queue_examples+batch_size)

    # Display the training images in the visualizer.
    tf.summary.image('images', images)

    return images, labels, i_paths


def gen_csv_paths(data_dir, pref, sessid = None):
    
    filenames = get_png_files(os.path.join(data_dir, 'images_' + pref))
    # segments = get_png_files(os.path.join(data_dir, 'segments_'+ pref))
    # Assuming that segments have just anno added to their names

    filenames.sort()
    segments = []
    for filename in filenames:
        name = filename
        name = name.replace("/images_","/segments_")
        segments.append(name[:-4]+"_anno"+name[-4:])
    all_files = np.array([filenames, segments])

    if pref == 'train':
        indices = [random.randint(0,len(all_files[0])-1) for i in range(len(all_files[0]))]
        all_files = all_files[:,indices]
        pd_arr = pd.DataFrame(all_files).transpose()
        pd_arr.to_csv(pref + str(sessid) + '.csv', index=False, header=False)
    else:
        pd_arr = pd.DataFrame(all_files).transpose()
        pd_arr.to_csv(pref + '.csv', index=False, header=False)


def get_read_input(eval_data, sessid = None):
    
    # Create queues that produce the filenames and labels to read.
    if eval_data == 'train':
        all_files_queue = tf.train.string_input_producer([eval_data + str(sessid) + '.csv'])
    else:
        all_files_queue = tf.train.string_input_producer([eval_data + '.csv'])

    # Read examples from files in the filename queue.
    read_input = read_dataset(all_files_queue)
    reshaped_image = tf.cast(read_input.uint8image, tf.float32)
    read_input.label = tf.cast(read_input.label, tf.int32)

    return read_input, reshaped_image


def get_png_files(dirname):
    return [dirname + '/' + f for f in os.listdir(dirname) if f.endswith('.png')]


def inputs(eval_data, batch_size, sessid):
    
    num_examples_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN
    read_input, reshaped_image = get_read_input(eval_data, sessid)

    # Image processing for evaluation.

    # Subtract off the mean and divide by the variance of the pixels.
    float_image = tf.image.per_image_standardization(reshaped_image)

    # Set the shapes of tensors.
    float_image.set_shape([IMAGE_HEIGHT, IMAGE_WIDTH, 3])
    read_input.label.set_shape([IMAGE_HEIGHT, IMAGE_WIDTH, 1])

    # Set max intensity to 1
    read_input.label = tf.cast(tf.divide(read_input.label, 255), tf.int32)

    # Ensure that the random shuffling has good mixing properties.
    min_fraction_of_examples_in_queue = 0.4
    min_queue_examples = int(num_examples_per_epoch *
                             min_fraction_of_examples_in_queue)

    # Generate a batch of images and labels by building up a queue of examples.
    shuffle = False
    # shuffle = False if eval_data == 'test' else True
    return _generate_image_and_label_batch(float_image, read_input.label, read_input.i_path,
                                           min_queue_examples, batch_size,
                                           shuffle=shuffle)

In [ ]:
# Builds the network
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import re

import tensorflow as tf
# from tensorflow.contrib.slim.python.slim.nets.resnet_v2 import bottleneck
import numpy as np
import math
import data_input

from tensorflow.python.ops import variable_scope
from tensorflow.contrib.layers.python.layers import utils
from tensorflow.contrib.layers.python.layers import layers
from tensorflow.python.ops import nn_ops
from tensorflow.contrib import layers as layers_lib
from tensorflow.contrib.slim.python.slim.nets import resnet_utils

FLAGS = tf.app.flags.FLAGS

# Basic model parameters.
tf.app.flags.DEFINE_integer('batch_size', 1)
tf.app.flags.DEFINE_string('data_dir', 'data')
tf.app.flags.DEFINE_boolean('use_fp16', False)
tf.app.flags.DEFINE_integer('num_layers', 6)
tf.app.flags.DEFINE_integer('num_classes', 2)
tf.app.flags.DEFINE_integer('feat_root', 32)
tf.app.flags.DEFINE_integer('deconv_root', 8)

# Global constants describing the BBBC006 data set.
IMAGE_WIDTH = data_input.IMAGE_WIDTH
IMAGE_HEIGHT = data_input.IMAGE_HEIGHT
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = data_input.NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = data_input.NUM_EXAMPLES_PER_EPOCH_FOR_EVAL

# Constants describing the training process.
MOVING_AVERAGE_DECAY = 0.9995  # The decay to use for the moving average.
NUM_EPOCHS_PER_DECAY = 100.0  # Epochs after which learning rate decays.
LEARNING_RATE_DECAY_FACTOR = 0.01  # Learning rate decay factor.
INITIAL_LEARNING_RATE = 0.05  # Initial learning rate.
DROPOUT_RATE = 0.5  # Probability for dropout layers.
S_CLASS_PROP = .2249  # Segments proportion of pixels in class 1.

# If a model is trained with multiple GPUs, prefix all Op names with tower_name
# to differentiate the operations. Note that this prefix is removed from the
# names of the summaries when visualizing a model.
TOWER_NAME = 'tower'


def _activation_summary(x):
	# Remove 'tower_[0-9]/' from the name in case this is a multi-GPU training
	# session. This helps the clarity of presentation on tensorboard.
	tensor_name = re.sub('%s_[0-9]*/' % TOWER_NAME, '', x.op.name)
	tf.summary.histogram(tensor_name + '/activations', x)
	tf.summary.scalar(tensor_name + '/sparsity', tf.nn.zero_fraction(x))


def _variable_on_cpu(name, shape, initializer):
	dtype = tf.float16 if FLAGS.use_fp16 else tf.float32
	var = tf.get_variable(name, shape, initializer=initializer, dtype=dtype)
	return var


def _variable_with_weight_decay(name, shape, stddev, wd):
	dtype = tf.float16 if FLAGS.use_fp16 else tf.float32
	var = _variable_on_cpu(
		name,
		shape,
		tf.truncated_normal_initializer(stddev=stddev, dtype=dtype))
	if wd is not None and not tf.get_variable_scope().reuse:
		weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
		tf.add_to_collection('losses', weight_decay)
	return var


def distorted_inputs():
	if not FLAGS.data_dir:
		raise ValueError('Please supply a data_dir')
	images, labels = data_input.distorted_inputs(batch_size=FLAGS.batch_size)
	if FLAGS.use_fp16:
		images = tf.cast(images, tf.float16)
		labels = tf.cast(labels, tf.float16)
	return images, labels


def inputs(eval_data, sessid):
	if not FLAGS.data_dir:
		raise ValueError('Please supply a data_dir')
	images, labels, i_paths = data_input.inputs(eval_data=eval_data,
										  batch_size=FLAGS.batch_size, sessid = sessid)
	labels = tf.cast(tf.divide(labels,255),tf.int32)
	if FLAGS.use_fp16:
		images = tf.cast(images, tf.float16)
		labels = tf.cast(labels, tf.float16)
	return images, labels, i_paths


def get_deconv_filter(shape):
	width = shape[0]
	height = shape[0]
	f = math.ceil(width / 2.0)
	c = (2.0 * f - 1 - f % 2) / (2.0 * f)

	bilinear = np.zeros([shape[0], shape[1]])
	for x in range(width):
		for y in range(height):
			bilinear[x, y] = (1 - abs(x / f - c)) * (1 - abs(y / f - c))

	weights = np.zeros(shape)
	for i in range(shape[2]):
		weights[:, :, i, i] = bilinear

	init = tf.constant_initializer(value=weights, dtype=tf.float32)
	return tf.get_variable(name='up_filter', initializer=init, shape=weights.shape)


def _deconv_layer(in_layer, w, b, dc, ds, scope):
	deconv = tf.nn.conv2d_transpose(in_layer, w, ds, strides=[1, dc, dc, 1],
									padding='SAME')
	deconv = tf.nn.bias_add(deconv, bias=b, name=scope.name)
	deconv = tf.nn.relu(deconv)
	_activation_summary(deconv)
	return deconv


def inference(images, train=True):
	feat_out = FLAGS.feat_root
	# in_layer = tf.layers.batch_normalization(images)
	in_layer = images

	# Deconvolution constant: kernel size = 2 * dc, stride = dc
	# Deconvolution output shape
	dc = FLAGS.deconv_root
	ds = [FLAGS.batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, FLAGS.num_classes]

	# Up-sampled layers 4-6 output maps for contours and segments, respectively
	s_outputs = []

	for layer in range(FLAGS.num_layers):
		# CONVOLUTION
		with tf.variable_scope('conv{}'.format(layer + 1)) as scope:
			# Double the number of feat_out for all but convolution layer 4
			feat_out *= 2 if layer != 4 else 1
			conv = tf.layers.conv2d(in_layer, feat_out, (3, 3), padding='same',
									activation=tf.nn.relu, name=scope.name)

			if train and layer > 3:  # During training, add dropout to layers 5 and 6
				conv = tf.nn.dropout(conv, keep_prob=DROPOUT_RATE)

			_activation_summary(conv)

		# POOLING
		# First and convolution layers has no pooling afterwards
		if 0 < layer:
			pool = tf.layers.max_pooling2d(conv, 2, 2, padding='same')

			_activation_summary(pool)
			in_layer = pool
		else:
			in_layer = conv

		# Transposed convolution and output mapping for segments and contours
		if layer > 2:  # Only applies to layers 3-5
			# TRANSPOSED CONVOLUTION
			with tf.variable_scope('deconv{0}'.format(layer + 1)) as scope:
				feat_in = in_layer.get_shape().as_list()[-1]
				shape = [dc * 2, dc * 2, FLAGS.num_classes, feat_in]
				w = get_deconv_filter(shape)
				b = _variable_on_cpu('biases', [FLAGS.num_classes],
							tf.constant_initializer(0.1))

				deconv = _deconv_layer(in_layer, w, b, dc, ds, scope)

			with tf.variable_scope('output{0}'.format(layer + 1)) as scope:
				output = tf.layers.conv2d(deconv, FLAGS.num_classes, (1, 1),
										  padding='same', activation=tf.nn.relu,
										  name=scope.name)
				s_outputs.append(output)
			dc *= 2
	s_fuse = tf.add_n(s_outputs)

	return s_fuse

"""
	Copied from tf website, just tweaked a bit to include BN and ReLU
"""
def bottleneck(inputs,
               depth,
               depth_bottleneck,
               stride,
               rate=1,
               outputs_collections=None,
               scope=None):
  
  with variable_scope.variable_scope(scope, 'bottleneck_v2', [inputs]) as sc:
    depth_in = utils.last_dimension(inputs.get_shape(), min_rank=4)
    preact = layers.batch_norm(
        inputs, activation_fn=nn_ops.relu, scope='preact')
    if depth == depth_in:
      shortcut = resnet_utils.subsample(inputs, stride, 'shortcut')
    else:
      shortcut = layers_lib.conv2d(
          preact,
          depth, [1, 1],
          stride=stride,
          normalizer_fn=None,
          activation_fn=None,
          scope='shortcut')

    residual = preact
    residual = tf.layers.batch_normalization(residual)
    residual = tf.nn.relu(residual)
    residual = layers_lib.conv2d(
        residual, depth_bottleneck, [1, 1], stride=1, scope='conv1')
    residual = tf.layers.batch_normalization(residual)
    residual = tf.nn.relu(residual)
    residual = resnet_utils.conv2d_same(
        residual, depth_bottleneck, 3, stride, rate=rate, scope='conv2')
    residual = tf.layers.batch_normalization(residual)
    residual = tf.nn.relu(residual)
    residual = layers_lib.conv2d(
        residual,
        depth, [1, 1],
        stride=1,
        normalizer_fn=None,
        activation_fn=None,
        scope='conv3')

    output = shortcut + residual

    return utils.collect_named_outputs(outputs_collections, sc.name, output)

def inference_bottleneck(images, train=True):
	in_layer = images
	feat_out = FLAGS.feat_root
	s_outputs = []
	
	with tf.variable_scope('bottleneck0-1') as scope:
		in_layer = tf.layers.max_pooling2d(in_layer, 2, 2, padding='same')
		in_layer = tf.layers.batch_normalization(in_layer)
		in_layer = tf.nn.relu(in_layer)
		in_layer = tf.layers.conv2d(in_layer, feat_out, (3, 3), padding='same', name=scope.name)
	with tf.variable_scope('bottleneck0-2') as scope:
		in_layer = tf.layers.batch_normalization(in_layer)
		in_layer = tf.nn.relu(in_layer)
		in_layer = tf.layers.conv2d(in_layer, feat_out, (3, 3), padding='same', name=scope.name)
		s_outputs.append(in_layer)
	for layer in range(5):
		with tf.variable_scope('bottleneck{0}-{1}'.format(layer + 1,1)) as scope:
			in_layer = tf.layers.max_pooling2d(in_layer, 2, 2, padding='same')
			in_layer = bottleneck(in_layer,min(feat_out*2,256),feat_out,1)
		with tf.variable_scope('bottleneck{0}-{1}'.format(layer + 1,2)) as scope:
			feat_out = min(feat_out*2,256)
			in_layer = bottleneck(in_layer,feat_out,feat_out,1)
			s_outputs.append(in_layer)
	# populate s_outputs
	encoding = in_layer
	dc = 2 
	ds = [FLAGS.batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, FLAGS.num_classes]
	for layer in range(len(s_outputs)):
		with tf.variable_scope('deconv{0}'.format(layer + 1)) as scope:
			in_layer = s_outputs[layer]
			feat_in = in_layer.get_shape().as_list()[-1]
			shape = [dc * 2, dc * 2, FLAGS.num_classes, feat_in]
			w = get_deconv_filter(shape)
			b = _variable_on_cpu('biases', [FLAGS.num_classes],
								 tf.constant_initializer(0.1))
			deconv = _deconv_layer(in_layer, w, b, dc, ds, scope)
			s_outputs[layer] = deconv
		dc *= 2

	s_fuse = tf.concat(s_outputs,axis=-1)
	with tf.variable_scope('after_fusion-1') as scope:
		s_fuse = tf.layers.batch_normalization(s_fuse)
		s_fuse = tf.nn.relu(s_fuse)
		s_fuse = tf.layers.conv2d(s_fuse, 2, (3, 3), padding='same', name=scope.name)
	with tf.variable_scope('after_fusion-2') as scope:
		s_fuse = tf.layers.batch_normalization(s_fuse)
		s_fuse = tf.nn.relu(s_fuse)
		s_fuse = tf.layers.conv2d(s_fuse, 2, (1, 1), padding='same', name=scope.name)
	return s_fuse, encoding

def _add_loss_summaries(total_loss):
	# Compute the moving average of all individual losses and the total loss.
	loss_averages = tf.train.ExponentialMovingAverage(0.9, name='avg')
	losses = tf.get_collection('losses')
	loss_averages_op = loss_averages.apply(losses + [total_loss])

	# Attach a scalar summary to all individual losses and the total loss; do the
	# same for the averaged version of the losses.
	for l in losses + [total_loss]:
		# Name each loss as '(raw)' and name the moving average version of the loss
		# as the original loss name.
		tf.summary.scalar(l.op.name + '_raw', l)
		tf.summary.scalar(l.op.name, loss_averages.average(l))

	return loss_averages_op


def train(total_loss, global_step):
	# Variables that affect learning rate.
	num_batches_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN / FLAGS.batch_size
	decay_steps = int(num_batches_per_epoch * NUM_EPOCHS_PER_DECAY)
	# Decay the learning rate exponentially based on the number of steps.
	lr = tf.train.exponential_decay(INITIAL_LEARNING_RATE,
									global_step,
									decay_steps,
									LEARNING_RATE_DECAY_FACTOR,
									staircase=True)
	tf.summary.scalar('learning_rate', lr)

	# Generate moving averages of all losses and associated summaries.
	loss_averages_op = _add_loss_summaries(total_loss)

	# Compute gradients.
	with tf.control_dependencies([loss_averages_op]):
		opt = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.2)
		grads = opt.compute_gradients(total_loss, var_list=tf.trainable_variables())

	# Apply gradients.
	apply_gradient_op = opt.apply_gradients(grads, global_step=global_step)

	# Add histograms for trainable variables.
	for var in tf.trainable_variables():
		tf.summary.histogram(var.op.name, var)

	# Add histograms for gradients.
	# for grad, var in grads:
	#     if grad is not None:
	#         tf.summary.histogram(var.op.name + '/gradients', grad)

	# Track the moving averages of all trainable variables.
	variable_averages = tf.train.ExponentialMovingAverage(
		MOVING_AVERAGE_DECAY, global_step)
	variables_averages_op = variable_averages.apply(tf.trainable_variables())

	with tf.control_dependencies([apply_gradient_op, variables_averages_op]):
		train_op = tf.no_op(name='train')

	return train_op


def get_show_preds(s_fuse):
	# Index 1 of fuse layers correspond to foreground, so discard index 0.
	_, s_logits = tf.split(tf.cast(tf.nn.softmax(s_fuse), tf.float32), 2, 3)

	tf.summary.image('s_logits', s_logits)
	return s_logits


def get_show_labels(labels):
	s_labels = labels
	s_labels = tf.cast(s_labels, tf.float32)

	tf.summary.image('s_labels', s_labels)
	return s_labels


def get_dice_coef(logits, labels):
	smooth = 1e-5
	inter = tf.reduce_sum(tf.multiply(logits, labels))
	l = tf.reduce_sum(logits)
	r = tf.reduce_sum(labels)
	return tf.reduce_mean((2.0 * inter + smooth) / (l + r + smooth))


def dice_op(s_fuse, labels):
	s_logits = get_show_preds(s_fuse)
	s_labels = get_show_labels(labels)

	s_dice = get_dice_coef(s_logits, s_labels)

	return s_dice

In [4]:
# generating csv

from data_input import gen_csv_paths

gen_csv_paths('reshaped_warwick/', 'train', 0)
gen_csv_paths('reshaped_warwick/', 'train', 1)
gen_csv_paths('reshaped_warwick/', 'train', 2)
gen_csv_paths('reshaped_warwick/', 'train', 3)
gen_csv_paths('reshaped_warwick/', 'test')
gen_csv_paths('reshaped_warwick/', 'train_eval')

In [1]:
# binary to train using a single GPU
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import time
import logging
import tensorflow as tf

import mainutils

FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('train_dir', '/home/lucifer/acadgild/project/assignement/warwick_train',
						   """Directory where to write event logs """
						   """and checkpoint.""")
tf.app.flags.DEFINE_string('eval_data', 'train',
						   """Either 'test' or 'train'.""")
tf.app.flags.DEFINE_integer('max_steps', 4000,
							"""Number of batches to run.""")
tf.app.flags.DEFINE_boolean('log_device_placement', False,
							"""Whether to log device placement.""")
tf.app.flags.DEFINE_integer('log_frequency', 200,
							"""How often to log results to the console.""")
tf.logging.set_verbosity(tf.logging.DEBUG)

def train(sessid):
	"""Train BBBC006 for a number of steps."""
	with tf.Graph().as_default():
		ckpt = tf.train.get_checkpoint_state(FLAGS.train_dir)
		global_step_init = -1
		global_step = tf.contrib.framework.get_or_create_global_step()
		if ckpt and ckpt.model_checkpoint_path:
			# Restores from checkpoint
			global_step_init = int(ckpt.model_checkpoint_path.split('/')[-1]
								   .split('-')[-1])

		# Get images and labels for BBBC006.
		# Force input pipeline to CPU:0 to avoid operations sometimes ending up
		# on GPU and resulting in a slow down.
		images, labels, i_paths = mainutils.inputs(eval_data=FLAGS.eval_data,sessid=sessid)
		# Build a Graph that computes the logits predictions from the
		# inference model.
		# s_fuse = mainutils.inference(images)
		s_fuse, encoding = mainutils.inference_bottleneck(images)
		mainutils.get_show_preds(s_fuse)

		# Calculate loss.
		segments_labels = labels
		with tf.variable_scope('{}_cross_entropy'.format('s')) as scope:
			class_prop = mainutils.S_CLASS_PROP
			weight_per_label = tf.scalar_mul(class_prop, tf.cast(tf.equal(segments_labels, 0),
																 tf.float32)) + \
							   tf.scalar_mul(1.0 - class_prop, tf.cast(tf.equal(segments_labels, 1),
																	   tf.float32))
			cross_entropy = tf.losses.sparse_softmax_cross_entropy(
				labels=tf.squeeze(segments_labels, squeeze_dims=[3]), logits=s_fuse)
			cross_entropy_weighted = tf.multiply(weight_per_label, cross_entropy)
			cross_entropy_mean = tf.reduce_mean(cross_entropy_weighted, name=scope.name)

		loss = cross_entropy_mean

		# Build a Graph that trains the model with one batch of examples and
		# updates the model parameters.
		train_op = mainutils.train(loss, global_step)

		class _LoggerHook(tf.train.SessionRunHook):
			"""Logs loss and runtime."""

			def begin(self):
				self._step = global_step_init
				self._start_time = time.time()

			def before_run(self, run_context):
				self._step += 1
				return tf.train.SessionRunArgs(loss)  # Asks for loss value.

			def after_run(self, run_context, run_values):
				if self._step % FLAGS.log_frequency == 0:
					current_time = time.time()
					duration = current_time - self._start_time
					self._start_time = current_time

					loss_value = run_values.results
					examples_per_sec = FLAGS.log_frequency * FLAGS.batch_size / duration
					sec_per_batch = float(duration / FLAGS.log_frequency)

					format_str = ('%s: step %d, loss = %.2f (%.1f examples/sec; %.3f '
								  'sec/batch)')
					print(format_str % (datetime.now(), self._step, loss_value,
										examples_per_sec, sec_per_batch))

		saver = tf.train.Saver()
		with tf.train.MonitoredTrainingSession(
				checkpoint_dir=FLAGS.train_dir,
				hooks=[tf.train.StopAtStepHook(last_step=FLAGS.max_steps),
					   tf.train.NanTensorHook(loss),
					   _LoggerHook()],
				config=tf.ConfigProto(
					log_device_placement=FLAGS.log_device_placement)
				) as mon_sess:
			ckpt = tf.train.get_checkpoint_state(FLAGS.train_dir)

			if ckpt:
				saver.restore(mon_sess, ckpt.model_checkpoint_path)
				logging.info("Model restored from file: %s" % ckpt.model_checkpoint_path)
			while not mon_sess.should_stop():
				_,losseval = mon_sess.run([train_op,loss])

def main(argv=None):  # pylint: disable=unused-argument
	FLAGS.train_dir = '/home/lucifer/acadgild/project/assignement/warwick_train_0'
	train(0)
	FLAGS.train_dir = '/home/lucifer/acadgild/project/assignement/warwick_train_1'
	train(1)
	FLAGS.train_dir = '/home/lucifer/acadgild/project/assignement/warwick_train_2'
	train(2)
	FLAGS.train_dir = '/home/lucifer/acadgild/project/assignement/warwick_train_3'
	train(3)

if __name__ == '__main__':
	tf.app.run()

W0912 17:24:50.424220 139864161822528 deprecation.py:323] From <ipython-input-1-df2c1ce4443c>:33: get_or_create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
W0912 17:24:50.429354 139864161822528 deprecation.py:323] From /home/lucifer/acadgild/project/assignement/data_input.py:148: string_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0912 17:24:50.434712 139864161822528 deprecation.py:323] From /home/lucifer/acadgild/project/test/testenv/lib/python3.7/site-packages/tensorflow/python/training/input.py:278: inp

W0912 17:24:52.497407 139864161822528 deprecation_wrapper.py:119] From /home/lucifer/acadgild/project/assignement/mainutils.py:450: The name tf.train.MomentumOptimizer is deprecated. Please use tf.compat.v1.train.MomentumOptimizer instead.

W0912 17:24:53.972949 139864161822528 deprecation.py:323] From /home/lucifer/acadgild/project/test/testenv/lib/python3.7/site-packages/tensorflow/python/training/moving_averages.py:433: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
I0912 17:24:57.157938 139864161822528 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0912 17:24:59.468716 139864161822528 monitored_session.py:240] Graph was finalized.
I0912 17:25:03.843045 139864161822528 session_manager.py:500] Running local_init_op.
I0912 17:25:03.968079 13986416182

2019-09-12 17:25:17.174074: step 0, loss = 0.41 (10.7 examples/sec; 0.094 sec/batch)


W0912 17:25:19.035815 139864161822528 basic_session_run_hooks.py:724] It seems that global step (tf.train.get_global_step) has not been increased. Current value (could be stable): 0 vs previous value: 0. You could increase the global step by passing tf.train.get_global_step() to Optimizer.apply_gradients or Optimizer.minimize.
I0912 17:26:24.239034 139864161822528 basic_session_run_hooks.py:692] global_step/sec: 1.49105
I0912 17:27:27.278023 139864161822528 basic_session_run_hooks.py:692] global_step/sec: 1.58632


2019-09-12 17:27:27.281356: step 200, loss = 0.36 (1.5 examples/sec; 0.651 sec/batch)


...

2019-09-12 20:19:20.798842: step 3800, loss = 0.03 (1.6 examples/sec; 0.609 sec/batch)


I0912 20:20:19.849335 139864161822528 basic_session_run_hooks.py:692] global_step/sec: 1.69336
I0912 20:21:19.278315 139864161822528 basic_session_run_hooks.py:692] global_step/sec: 1.68268
I0912 20:21:19.284389 139864161822528 basic_session_run_hooks.py:606] Saving checkpoints for 4000 into /home/lucifer/acadgild/project/assignement/warwick_train_3/model.ckpt.


2019-09-12 20:21:19.283760: step 4000, loss = 0.06 (1.7 examples/sec; 0.592 sec/batch)


SystemExit: 

/home/lucifer/acadgild/project/test/testenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
# Evaluation for the traind model
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import math
import time
from PIL import Image
import numpy as np
import tensorflow as tf

import mainutils

FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('eval_dir', '/home/lucifer/acadgild/project/assignement/warwick_eval')
tf.app.flags.DEFINE_string('eval_data', 'train_eval')
tf.app.flags.DEFINE_string('checkpoint_dir', '/home/lucifer/acadgild/project/assignement/warwick_train')
tf.app.flags.DEFINE_integer('eval_interval_secs', 60 * 5)
tf.app.flags.DEFINE_integer('num_examples', 100)
tf.app.flags.DEFINE_boolean('run_once', True)


def eval_once(saver, dice_op, summary_writer, summary_op, s_fuse, images, labels, i_paths, encoding, sessid):

	FLAGS.checkpoint_dir = '/home/lucifer/acadgild/project/assignement/warwick_train_'+str(sessid)
	with tf.Session() as sess:
		ckpt = tf.train.get_checkpoint_state(FLAGS.checkpoint_dir)
		if ckpt and ckpt.model_checkpoint_path:
			# Restores from checkpoint
			saver.restore(sess, ckpt.model_checkpoint_path)
			print("Model restored from file: %s" % ckpt.model_checkpoint_path)
			global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
		else:
			print('No checkpoint file found')
			return

		# Start the queue runners.
		coord = tf.train.Coordinator()
		predictions = {}
		encodings = {}
		dice_scores = {}
		try:
			threads = []
			for qr in tf.get_collection(tf.GraphKeys.QUEUE_RUNNERS):
				threads.extend(qr.create_threads(sess, coord=coord, daemon=True,
												 start=True))

			num_iter = int(math.ceil(FLAGS.num_examples / FLAGS.batch_size))
			avg_s_dice = 0
			step = 0
			while step < num_iter and not coord.should_stop():
				s_dice,i_path,s_fuse_out,encoded_image = sess.run([dice_op,i_paths,s_fuse,encoding])
				predictions[i_path[0]] = s_fuse_out
				encodings[i_path[0]] = encoded_image
				dice_scores[i_path[0]] = s_dice
				im0 = s_fuse_out[0,:,:,0]
				im1 = s_fuse_out[0,:,:,1]
				image = (im1>im0)*128
				im = Image.fromarray(image.astype(np.uint8))
				im.save('results/'+i_path[0].split('/')[2]+'.bmp')
				avg_s_dice += s_dice
				step += 1

			avg_s_dice /= step
			print('%s: s_dice avg = %.3f' % (datetime.now(), avg_s_dice))

			summary = tf.Summary()
			summary.ParseFromString(sess.run(summary_op))
			summary.value.add(tag='dice_s', simple_value=avg_s_dice)
			summary_writer.add_summary(summary, global_step)
		except Exception as e:  # pylint: disable=broad-except
			coord.request_stop(e)
		if FLAGS.eval_data == 'train_eval':
			np.save('train_eval_data_'+str(sessid)+'.npy',[predictions, encodings, dice_scores]) # otherwise don't save
		coord.request_stop()
		coord.join(threads, stop_grace_period_secs=20)


def evaluate():
	"""Eval BBBC006 for a number of steps."""
	for sessid in range(4):
		FLAGS.eval_dir = '/home/lucifer/acadgild/project/assignement/warwick_eval_' + str(sessid)
		with tf.Graph().as_default() as g:
			# Get images and labels for BBBC006.
			images, labels, i_paths = mainutils.inputs(eval_data=FLAGS.eval_data, sessid=sessid)
			
			# Build a Graph that computes the logits predictions from the
			# inference model.
			# s_fuse = mainutils.inference(images, train=False)
			s_fuse, encoding = mainutils.inference_bottleneck(images, train=False)

			dice_op = mainutils.dice_op(s_fuse, labels)

			# Restore the moving average version of the learned variables for eval.
			variable_averages = tf.train.ExponentialMovingAverage(
				mainutils.MOVING_AVERAGE_DECAY)
			variables_to_restore = variable_averages.variables_to_restore()
			saver = tf.train.Saver(variables_to_restore)

			# Build the summary operation based on the TF collection of Summaries.
			summary_op = tf.summary.merge_all()
			summary_writer = tf.summary.FileWriter(FLAGS.eval_dir, g)
			s_fuse_softmax = tf.nn.softmax(s_fuse)

			while True:
				eval_once(saver, dice_op, summary_writer, summary_op, s_fuse_softmax, images, labels, i_paths, encoding, sessid)
				if FLAGS.run_once:
					break
				time.sleep(FLAGS.eval_interval_secs)


def main(argv=None):  # pylint: disable=unused-argument
	if tf.gfile.Exists(FLAGS.eval_dir):
		tf.gfile.DeleteRecursively(FLAGS.eval_dir)
	tf.gfile.MakeDirs(FLAGS.eval_dir)
	evaluate()


if __name__ == '__main__':
	tf.app.run()

TypeError: DEFINE_string() missing 1 required positional argument: 'help'